<table>
  <tr>
    <td><img src="https://github.com/rvss-australia/RVSS/blob/main/Pics/RVSS-logo-col.med.jpg?raw=1" width="400"></td>
    <td><div align="left"><font size="30">Planar homography</font></div></td>
  </tr>
</table>

(c) Peter Corke 2024

Robotics, Vision & Control: Python, see section 13.6.2

## Configuring the Jupyter environment
We need to import some packages to help us with linear algebra (`numpy`), graphics (`matplotlib`), and machine vision (`machinevisiontoolbox`).
If you're running locally you need to have these packages installed.  If you're running on CoLab we have to first install machinevisiontoolbox which is not preinstalled, this will be a bit slow.

In [ ]:
try:
    import google.colab
    print('Running on CoLab')
    !pip install machinevision-toolbox-python
    COLAB = True
except:
    COLAB = False
    
%matplotlib inline

import numpy as np
np.set_printoptions(linewidth=120, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

from spatialmath import SE3
from spatialmath.base import e2h, h2e, homtrans, plot_sphere
from machinevisiontoolbox import CentralCamera


***

We define a central perspective camera, up high, looking obliquely downward at the ground

In [ ]:
camera = CentralCamera(f=0.012, rho=10e-6, imagesize=1000, 
        pose=SE3(0, 0, 8) * SE3.Rx(-2.8))

And we can plot the camera in the 3D world

In [ ]:
ax = camera.plot(scale=2, shape='camera', color='k', frame=True)
ax.set_xlim(-8, 12)
ax.set_ylim(-10, 10)
ax.set_zlim(0, 10)

A shape on the ground plane is defined by a set of 2D coordinates

In [ ]:
P = np.column_stack([[-1, 1], [-1, 2], [ 2,2], [2, 1]])
P

We can project the ground plane points onto the image plane by making them 3D points, we augment each column with a zero, since the ground plane is defined by $z=0$

In [ ]:
camera.project_point(np.vstack([P, np.zeros((4,))]))

The homography is computed from the camera matrix by deleting column two (the z column)

In [ ]:
H = np.delete(camera.C(), 2, axis=1)
H

We can use this matrix to directly compute the image plane points, by transforming the homogeneous ground plane points

In [ ]:
h2e(H @ e2h(P))

or more simply

In [ ]:
homtrans(H, P)

which first converts `P` to homogeneous form, performs the multiplication, then converts the resulting homogeneous coordinates to Euclidean.


H is square and of full rank, so it is invertible. This means that we can perform the inverse mapping, from the image plane
to the ground plane.

The camera has a 1000 x 1000 image plane so the coordinates of its corners are


In [ ]:
p = np.column_stack([[0, 0], [0, 1000], [1000, 1000], [1000, 0]])
p

and on the ground plane these are the points

In [ ]:
Pi = homtrans(np.linalg.inv(H), p)
Pi

which we can overlay the camera's field of view (the corners of the image plane) onto the "world view" of the imaging setup that we showed earlier

In [ ]:
ax = camera.plot(scale=2, shape='camera', color='k', frame=True)
k = [0, 1, 2, 3, 0]
ax.plot(Pi[0, k], Pi[1, k], np.zeros(5), 'b--')
